### IMPORTING FILES AND LIBRARIES

In [11]:
import pandas as pd 
import numpy as np 
import xgboost as xgb

In [12]:
df = pd.read_csv("/Users/lalitaneeharikavajjhala/Desktop/KAARA/TESLA PREDICTION/Tesla Dataset.csv")

### PREPROCESSING

In [13]:
df.head

<bound method NDFrame.head of             Date        Open        High         Low       Close   Adj Close  \
0     2010-06-29    1.266667    1.666667    1.169333    1.592667    1.592667   
1     2010-06-30    1.719333    2.028000    1.553333    1.588667    1.588667   
2     2010-07-01    1.666667    1.728000    1.351333    1.464000    1.464000   
3     2010-07-02    1.533333    1.540000    1.247333    1.280000    1.280000   
4     2010-07-06    1.333333    1.333333    1.055333    1.074000    1.074000   
...          ...         ...         ...         ...         ...         ...   
3504  2024-05-31  178.500000  180.320007  173.820007  178.080002  178.080002   
3505  2024-06-03  178.130005  182.639999  174.490005  176.289993  176.289993   
3506  2024-06-04  174.779999  177.759995  174.000000  174.770004  174.770004   
3507  2024-06-05  175.350006  176.149994  172.130005  175.000000  175.000000   
3508  2024-06-06  174.419998  176.919998  172.729996  175.440002  175.440002   

         

In [14]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [15]:
features = ['Open', 'High', 'Low']  
target_close = 'Close'

### MODEL BUILDING

In [16]:
model_close = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [17]:
X_close = df[features]
y_close = df[target_close]
model_close.fit(X_close, y_close)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [18]:
def predict_close_price(date, open_price, high_price, low_price):
    # Preparing the feature vector for prediction
    features_for_date = pd.DataFrame({
        'Open': open_price,
        'High': high_price,
        'Low': low_price
    }, index=[date])

    # Concatenating with existing data to ensure the model has all necessary columns
    X_pred_close = pd.concat([df[features], features_for_date[features]])

    # Predicting using the model
    predicted_close = model_close.predict(X_pred_close.iloc[-1].values.reshape(1, -1))[0]

    return predicted_close

### PREDICTION

In [19]:
date_to_predict = '2024-01-10'
open_price = 199.0
high_price = 195.0
low_price = 105.0

In [20]:
date = pd.to_datetime(date_to_predict)
predicted_close = predict_close_price(date, open_price, high_price, low_price)
print(f'Predicted Close price for given values is: {predicted_close}')

Predicted Close price for given values is: 154.08834838867188


### EVALUATION OF THE MODEL

In [21]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Function to evaluate the model performance
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, r2, mae

y_pred_close = model_close.predict(X_close)

mse_close, r2_close, mae_close = evaluate_model(y_close, y_pred_close)

# Printing the evaluation results
print(f"Mean Squared Error (MSE) on test set: {mse_close}")
print(f"R-squared (R²) Score on test set: {r2_close}")
print(f"Mean Absolute Error (MAE) on test set: {mae_close}")


Mean Squared Error (MSE) on test set: 2.2712822060375726
R-squared (R²) Score on test set: 0.9997812688923359
Mean Absolute Error (MAE) on test set: 0.5893805569634013
